In [0]:
%run "/Workspace/Projects/formula1/Data Ingestion/Prod/01-setup"

In [0]:
%run "/Workspace/Projects/formula1/Data Ingestion/Prod/02-common_functions"

In [0]:
race_df=spark.read.parquet(f"{processed_folder_path}/races").withColumnRenamed("year_id","race_year")\
                     .withColumnRenamed("race_timestamp","race_time").withColumnRenamed("name","race_name").withColumnRenamed("ingestion_date","race_date")


In [0]:
circuit_df=spark.read.parquet(f"{processed_folder_path}/circuits").withColumnRenamed("location","circuit_location") 

In [0]:
drivers_df=spark.read.parquet(f"{processed_folder_path}/drivers").withColumnRenamed("name","driver_name").withColumnRenamed("nationality","driver_nationality").withColumnRenamed("number","driver_number")

In [0]:
constructors_df=spark.read.parquet(f"{processed_folder_path}/constructors") 

In [0]:
results_df=spark.read.parquet(f"{processed_folder_path}/results") 

Join race and circuits

In [0]:
circuit_race_df=circuit_df.join(race_df,circuit_df.circuit_id==race_df.circuit_id,"inner")\
                .select(race_df.race_id, race_df.race_year,race_df.race_name,race_df.race_date,circuit_df.circuit_id,circuit_df.circuit_location)


Join results to all other dotaframes  

In [0]:
race_results_df=results_df.join(circuit_race_df,results_df.race_id==circuit_race_df.race_id,"inner")\
                .join(drivers_df,results_df.driver_id==drivers_df.driver_id,"inner")\
                .join(constructors_df,results_df.constructor_id==constructors_df.constructor_id,"inner")

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df=race_results_df.select("race_year","race_name","race_date","circuit_id","circuit_location","driver_ref","driver_number","driver_nationality","driver_name", "grid","fastest_lap","points","position").withColumn("created_date",current_timestamp())

In [0]:
final_df.write.mode("overwrite").saveAsTable("f1_processed.race_results")

In [0]:
final_df.write.mode("overwrite").parquet(f"{output_folder_path}/final")
dbutils.notebook.exit("Success")